In [1]:
# ------------>>>>>>>> RUN THIS CODE CELL <<<<<<<<------------
# === CELL TYPE: IMPORTS AND SETUP

import datetime
import math

import numpy as np
import pandas as pd

## First we need to open the Dataframes


In [2]:
games_df = pd.read_csv("./games_data_table.csv").drop("Unnamed: 0", axis=1)
players_df = pd.read_csv("./players_data_table.csv").drop("Unnamed: 0", axis=1)

In [3]:
games_df_main = games_df.copy()
games_df.drop_duplicates("match_id", inplace=True)

### We need to extract some data from the tables and combine them

#### get player id


In [4]:
def get_player_id(player_name):
    try:
        row = players_df[players_df["name"] == player_name]
        return row["id"].values[0]
    except:
        return None

#### get the dataframe that represents previous matches between the two players

In [5]:
def get_vs_df(p1, p2):
    df = games_df[(games_df["p1_id"] == p1) | (games_df["p2_id"] == p1)]
    df = df[(df["p1_id"] == p2) | (df["p2_id"] == p2)]
    return df

In [6]:
test = get_vs_df(745, 675)
test

,match_id,match_type,date,p1_id,p1_name,p1_club,p1_rank,p1_sets,p1_home,p1_points_gained,p2_id,p2_name,p2_club,p2_rank,p2_sets,p2_home,p2_points_gained,winner_id
5,327694,ליגת על גברים 2022-2023,2022-11-15,745,יונתן שוסטרמן,מ. בני הרצליה,1764.3,3,1.0,4.5,675,רון דימנט,הישגי כרמיאל,1444.4,0,0.0,-3.00,745
3411,127066,ליגות גברים 2017-2018,2017-10-24,675,רון דימנט,הישגי כרמיאל,1328.0,0,0.0,-3.0,745,יונתן שוסטרמן,הפועל חיפה,1624.6,3,1.0,4.50,745
3433,108068,משחקי דירוג סבב 3,2017-02-20,675,רון דימנט,הישגי כרמיאל,1291.1,0,NaN,-4.5,745,יונתן שוסטרמן,מכבי זאב זכרון יעקב,1415.9,3,NaN,6.75,745
3441,102955,ליגות גברים 2016-2017,2017-01-24,675,רון דימנט,הישגי כרמיאל,1288.3,1,1.0,-4.5,745,יונתן שוסטרמן,מכבי זאב זכרון יעקב,1427.9,3,0.0,6.75,745
3447,98223,משחקי דירוג סבב 2,2016-12-29,675,רון דימנט,הישגי כרמיאל,1213.3,3,NaN,36.0,745,יונתן שוסטרמן,מכבי זאב זכרון יעקב,1421.7,2,NaN,-24.00,675
3458,96043,ליגות נוער 2016-2017,2016-11-30,675,רון דימנט,הישגי כרמיאל,1244.8,1,0.0,-4.8,745,יונתן שוסטרמן,מכבי זאב זכרון יעקב,1317.0,3,1.0,5.76,745
3461,95000,ליגות גברים 2016-2017,2016-11-15,675,רון דימנט,הישגי כרמיאל,1244.8,2,0.0,-6.0,745,יונתן שוסטרמן,מכבי זאב זכרון יעקב,1317.0,3,1.0,9.00,745


#### get the form of each player 

By calling this function we are trying to acquire the form of the player that is represented by the matches of the week before.
It sums the total point he recieved form the previous games earlier today up to a week eralier.
We can use the form to determine if he is in a good shape or not before each match

In [7]:
def get_player_form(player_id, date, match_id):
    form = 0
    try:
        datetime_object = datetime.datetime.strptime(date, "%Y-%m-%d")
        start = datetime_object - datetime.timedelta(days=14)
        end = datetime_object
        date_time_start = start.strftime("%Y-%m-%d")
        date_time_end = end.strftime("%Y-%m-%d")
        temp_df = games_df[games_df["date"] <= date_time_end]
        temp_df.drop_duplicates("match_id", inplace=True)
        temp_df = temp_df[temp_df["date"] >= date_time_start]
        temp_df = temp_df[temp_df["match_id"] != match_id]
        temp_df = temp_df[
            (temp_df["p1_id"] == player_id) | (temp_df["p2_id"] == player_id)
        ]
        for j, row in temp_df.iterrows():
            form += (
                row["p1_points_gained"]
                if row["p1_id"] == player_id
                else row["p2_points_gained"]
            )
    except:
        pass
    return form

#### get the win history of each player against each other 


In [8]:
def get_wins_history(vs_df, date):
    vs_df = vs_df[vs_df["date"] < date].copy()
    p1 = {"id": vs_df["p1_id"].unique()[0]}
    p2 = {"id": vs_df["p2_id"].unique()[0]}
    p1_wins = 0
    p2_wins = 0
    p1["wins"] = len(vs_df[vs_df["winner_id"] == p1["id"]])
    p2["wins"] = len(vs_df[vs_df["winner_id"] == p2["id"]])
    return p1, p2

In [9]:
print(get_wins_history(test, "2022-11-16"))

({'id': 745, 'wins': 6}, {'id': 675, 'wins': 1})


After writing the relevant functions we can now use and update the relevant DataFrame

In [10]:
games_dict = games_df_main.to_dict("series")
games_dict["p1_prevwins"] = []
games_dict["p2_prevwins"] = []
games_dict["p1_form"] = []
games_dict["p2_form"] = []

#### Updating the DataFrame by calling this function

In [11]:
def wins_calc():
    for i, row in games_df_main.iterrows():

        p1_id = row["p1_id"]
        p2_id = row["p2_id"]
        vs_df = get_vs_df(p1_id, p2_id)
        try:
            wins = get_wins_history(vs_df, row["date"])
            games_dict["p1_prevwins"].append(
                int(wins[0]["wins"]) if wins[0]["id"] == p1_id else int(wins[1]["wins"])
            )
            games_dict["p2_prevwins"].append(
                int(wins[0]["wins"]) if wins[0]["id"] == p2_id else int(wins[1]["wins"])
            )
        except:
            games_dict["p1_prevwins"].append(0)
            games_dict["p2_prevwins"].append(0)

In [12]:
wins_calc()
print("Wins calc Done")

Wins calc Done


In [33]:
for i, row in games_df_main.iterrows():
    p1_id = row["p1_id"]
    p2_id = row["p2_id"]
    date = row["date"]
    match_id = row["match_id"]
    p1_form = get_player_form(p1_id, date, match_id)
    p2_form = get_player_form(p2_id, date, match_id)
    games_dict["p1_form"].append(p1_form)
    games_dict["p2_form"].append(p2_form)

C:\Users\Shlomi\AppData\Local\Temp\ipykernel_5720\117951440.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.drop_duplicates("match_id", inplace=True)


In [34]:
print("Forms Done")

Forms Done


In [16]:
games_df = pd.DataFrame.from_dict(games_dict)

In [17]:
games_df.to_csv("games_processed_table.csv", sep=",", encoding="utf-8-sig")